In [16]:
# Import drive, numpy, and pandas packages that we need
from google.colab import drive
import numpy as np
import pandas as pd
# Mount our drive and the directory to where I keep my FW458 files
drive.mount('/content/drive')
filedir = '/content/drive/MyDrive/FW458/'
filename = filedir + "Wcr_GPPdaily.csv"
# Read CSV file
DF = pd.read_csv(filename)
# Display first few rows
print(DF.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   TIMESTAMP    TA_F  SW_IN_F  VPD_F  GPP_NT_VUT_REF
0   19990101 -20.063   72.603  0.413       -0.517364
1   19990102 -12.814   12.358  0.147       -0.094241
2   19990103 -12.625   33.132  0.128       -0.166819
3   19990104 -18.652   93.481  0.263       -0.582301
4   19990105 -20.269   45.502  0.261       -0.568240


In [17]:
# Data preprocessing
# Rename columns
DF.rename(columns={'TA_F': 'TA', 'SW_IN_F': 'SW', 'VPD_F': 'VPD', 'GPP_NT_VUT_REF': 'GPP'}, inplace=True)
# Drop TIMESTAMP column
DF.drop(columns=['TIMESTAMP'], inplace=True)
# Drop rows with any missing values to ensure data consistency
DF.dropna(inplace=True)
print(DF.head())
print('Interestingly, none of rows have any missing values so dataframe has stayed the same size')

       TA      SW    VPD       GPP
0 -20.063  72.603  0.413 -0.517364
1 -12.814  12.358  0.147 -0.094241
2 -12.625  33.132  0.128 -0.166819
3 -18.652  93.481  0.263 -0.582301
4 -20.269  45.502  0.261 -0.568240
Interestingly, none of rows have any missing values so dataframe has stayed the same size


In [18]:
# Create interaction terms
DF['SW_VPD'] = DF['SW'] * DF['VPD']
DF['SW_TA'] = DF['SW'] * DF['TA']
DF['VPD_TA'] = DF['VPD'] * DF['TA']
print(DF.head())

       TA      SW    VPD       GPP     SW_VPD        SW_TA    VPD_TA
0 -20.063  72.603  0.413 -0.517364  29.985039 -1456.633989 -8.286019
1 -12.814  12.358  0.147 -0.094241   1.816626  -158.355412 -1.883658
2 -12.625  33.132  0.128 -0.166819   4.240896  -418.291500 -1.616000
3 -18.652  93.481  0.263 -0.582301  24.585503 -1743.607612 -4.905476
4 -20.269  45.502  0.261 -0.568240  11.876022  -922.280038 -5.290209


In [19]:
# Import scipy's minimize function
from scipy.optimize import minimize

# Define the linear model equation to be minimized: GPP=β0​+β1​⋅SW+β2​⋅VPD+β3​⋅TA+β4​⋅(SW×VPD)+β5​⋅(SW×TA)+β6​⋅(VPD×TA)
def costfunc(params, SW, VPD, TA, SW_VPD, SW_TA, VPD_TA, GPP):
    B0, B1, B2, B3, B4, B5, B6 = params

    GPPpred = B0 + B1*SW + B2*VPD + B3*TA + B4*SW_VPD + B5*SW_TA + B6*VPD_TA

    # use MSE as cost function
    return (np.mean((GPPpred - GPP)**2))
# Define variables that hold each column
SW = DF['SW'].values
VPD = DF['VPD'].values
TA = DF['TA'].values
SW_VPD = DF['SW_VPD'].values
SW_TA = DF['SW_TA'].values
VPD_TA = DF['VPD_TA'].values
GPP = DF['GPP'].values
# Make initial guesses of Beta coefficients
guess = [10, 10, 10, 10, 10, 10, 10]
# Run scipy's minimization function on our cost function of MSE for the linear model
mymin = minimize(costfunc, guess, args=(SW, VPD, TA, SW_VPD, SW_TA, VPD_TA, GPP))
# Get the optimized parameters
B0, B1, B2, B3, B4, B5, B6 = mymin.x
# Print each optimized parameter
print('B0 =', B0)
print('B1 =', B1)
print('B2 =', B2)
print('B3 =', B3)
print('B4 =' , B4)
print('B5 =', B5)
print('B6 =', B6)

B0 = -944.2399476092163
B1 = -3.24799110445965
B2 = -648.5364837803058
B3 = 82.16771903366768
B4 = 1.7922540924246166
B5 = -0.45606529845713484
B6 = 13.59672262795932


In [22]:
from sklearn.linear_model import LinearRegression
# Define Independent (X) and Dependent (y) Variables z
X = DF[['SW', 'VPD', 'TA', 'SW_VPD', 'VPD_TA', 'SW_TA']] # X variable has SW, VPD, TA, SW_VPD, VPD_TA, SW_TA
y = DF['GPP'] # Y variable is GPP
# Fit the model
model = LinearRegression()
model.fit(X, y)
# Print coefficient and intercepts
print("Coefficients:", model.coef_)  # Slopes for each feature
print("Intercept:", model.intercept_)  # Bias term

Coefficients: [-3.24914968e+00 -6.48574173e+02  8.21699813e+01  1.79249151e+00
  1.35954025e+01 -4.56050357e-01]
Intercept: -944.0672018206467


In [24]:
# Computer r-squared
# To do this, I will use the formula r^2 = 1 - residual sum of squares / total sum of squares
# FIrst I will do this for the scipy function
# Predict GPP using optimized parameters
GPPpred = (B0 + B1*SW + B2*VPD + B3*TA + B4*SW*VPD + B5*SW*TA + B6*VPD*TA)
SS_res = np.sum((GPP - GPPpred)**2)
SS_tot = np.sum((GPP - np.mean(GPP))**2)

R2_scipy = 1 - (SS_res / SS_tot)

# Now I will get R_squared of the sklearn model by using its model.score function
# Compute R_squarted
R2_sklearn = model.score(X, y)

print("R-squared of scipy model =", R2_scipy)
print("R-squared of sklearn model =", R2_sklearn)
print("As can be seen, the R-squared are pratically identical (except for numerical errors), which is as expected since they are both identical models, just different functions")

R-squared of scipy model = 0.013582700652710278
R-squared of sklearn model = 0.01358270093325531
As can be seen, the R-squared are pratically identical (except for numerical errors), which is as expected since they are both identical models, just different functions
